In [13]:
"""
Defines one instance of a genome, which is a collection of genes and a fitness score.
 - Genes are a list of weights that are applied to each indicator to dictate whether to buy or sell.
 - Fitness is a score that is calculated by the fitness function, which is the total profit of the genome on BTC.
"""
from ta.volatility import BollingerBands
import numpy as np
import pandas as pd
import random

class Genome():
	def __init__(self, fitness, buy_genes, sell_genes):
		self.fitness = fitness
		self.buy_genes = buy_genes
		self.sell_genes = sell_genes

	def __str__(self):
		return "Genome: fitness = {}, buy_genes = {}, sell_genes = {}".format(self.fitness, self.buy_genes, self.sell_genes)

	def __repr__(self):
		return self.__str__()

class Dataset():
	def __init__(self, data):
		self.data = data
		self.bollingerbands()
		self.save()
	
	def bollingerbands(self):
	
		short_window = 10 ### optimization target
		medium_window = 50 ### optimization target

		indicator_bb = BollingerBands(close=self.data["close"], window=short_window, window_dev=2)
		indicator_bb_m = BollingerBands(close=self.data["close"], window=medium_window, window_dev=2)
		df_temp = pd.DataFrame()

		# Add Bollinger Bands features
		df_temp['mavg_s'] = indicator_bb.bollinger_mavg()
		df_temp['mavg_m'] = indicator_bb_m.bollinger_mavg()

		self.data["bollinger_buy"] = np.where(df_temp['mavg_m'] > df_temp['mavg_s'], 1,-1)
		self.data["bollinger_sell"] = np.where(df_temp['mavg_m'] < df_temp['mavg_s'], 1,-1)
	
	def save(self):
		self.data.to_csv("processed_data.csv")
	
	def __str__(self):
		return "Dataset: size = {}".format(len(self.data))

	def __repr__(self):
		return self.__str__()

class BuyIndicators():
	def __init__(self):
		self.indicators = ["bollinger_buy"]
		self.size = len(self.indicators)

	def __str__(self):
		return "BuyIndicators: size = {}, indicators = {}".format(self.size, self.indicators)

	def __repr__(self):
		return self.__str__()

class SellIndicators():
	def __init__(self):
		self.indicators = ["bollinger_sell"]
		self.size = len(self.indicators)

	def __str__(self):
		return "SellIndicators: size = {}, indicators = {}".format(self.size, self.indicators)

	def __repr__(self):
		return self.__str__()


Dataset(pd.read_csv("raw_data.csv"))

Dataset: size = 719

In [48]:
class TrainGenomes():
	def __init__(self, population_size, data):
		self.population_size = population_size
		self.data = data
		self.buy_genes = BuyIndicators()
		self.sell_genes = SellIndicators()

	def train(self):
		# Initialize the population
		population = self.initialize_population(self.population_size, self.buy_genes, self.sell_genes)
		# Evaluate the population
		population = self.evaluate_population(population)
		# Sort the population by fitness
		population.sort(key=lambda x: x.fitness, reverse=True)
		# # Keep track of the best genome
		best_genome = population[0]
		# # Keep track of the number of generations with no improvement
		# no_improvement = 0
		# # Keep track of the number of generations
		# generations = 0
		# # Keep track of the time
		# start_time = time.time()
		# Keep track of the best fitness
	
	def initialize_population(self, population_size, buy_genes, sell_genes):
		population = []
		for i in range(population_size):
			buy_genes = np.random.uniform(-1,1,buy_genes.size)
			sell_genes = np.random.uniform(-1,1,sell_genes.size)
			population.append(Genome(0, buy_genes, sell_genes))

		return population
	
	def evaluate_population(self, population):
		for genome in population:
			genome.fitness = self.fitness(genome)
		return population
	
	def fitness(self, genome):
		buying = True
		fiat_money = 100
		btc_money = 0
		fee = 0.02 # Each buy or sell event costs 2% of current holdings.
		
		#Create a buy and sell dataframe using the genomes weights and the indicators status
		temp_df = pd.DataFrame()
		for i,indicator in enumerate(self.buy_genes.indicators):
			try:
				temp_df["buy"] += self.data[indicator] * genome.buy_genes[i]
			except KeyError:
				temp_df["buy"] = self.data[indicator] * genome.buy_genes[i]

		for i,indicator in enumerate(self.sell_genes.indicators):
			try:
				temp_df["sell"] += self.data[indicator] * genome.sell_genes[i]
			except KeyError:
				temp_df["sell"] = self.data[indicator] * genome.sell_genes[i]
		
		for index, row in self.data.iterrows():
			if buying:
				if temp_df.iloc[index]["buy"] > 0:
					btc_money = ( fiat_money / row['close'] ) * (1 - fee)
					fiat_money = 0
					buying = False
					print(f"BUYING: Day:{index},  Date: {row['timestamp']}")
			elif temp_df.iloc[index]["sell"] > 0:
					fiat_money = (btc_money * row['close'] ) * (1 - fee)
					btc_money = 0
					buying = True
					print(f"SELLING: Day:{index},  Date: {row['timestamp']}")
		
		return fiat_money + (btc_money * self.data.iloc[-1]['close'])


	# def select_parents(self, population):
	# 	# Select the parents using tournament selection
	# 	parent1 = self.tournament_selection(population)
	# 	parent2 = self.tournament_selection(population)
	# 	return parent1, parent2
	
	# def tournament_selection(self, population):
	# 	# Select two random genomes
	# 	genome1 = population[np.random.randint(len(population))]
	# 	genome2 = population[np.random.randint(len(population))]
	# 	# Return the fitter genome
	# 	return genome1 if genome1.fitness > genome2.fitness else genome2
	
	# def crossover(self, parent1, parent2):
	# 	# Select a crossover point
	# 	crossover_point = np.random.randint(1, parent1.size)
	# 	# Create the offspring
	# 	offspring1 = Genome(parent1.size, 0, np.concatenate((parent1.genes[:crossover_point], parent2.genes[crossover_point:])))
	# 	offspring2 = Genome(parent1.size, 0, np.concatenate((parent2.genes[:crossover_point], parent1.genes[crossover_point:])))
	# 	return offspring1, offspring2

	# def mutate(self, genome):
	# 	# Select a mutation point
	# 	mutation_point = np.random.randint(genome.size)
	# 	# Mutate the genome
	# 	genome.genes[mutation_point] = 1 if genome.genes[mutation_point] == 0 else 0
	# 	return genome

TrainGenomes(1, pd.read_csv("processed_data.csv")).train()

BUYING: Day:49,  Date: 2021-06-25
SELLING: Day:50,  Date: 2021-06-26
BUYING: Day:51,  Date: 2021-06-27
SELLING: Day:52,  Date: 2021-06-28
BUYING: Day:53,  Date: 2021-06-29
SELLING: Day:54,  Date: 2021-06-30
BUYING: Day:55,  Date: 2021-07-01
SELLING: Day:56,  Date: 2021-07-02
BUYING: Day:57,  Date: 2021-07-03
SELLING: Day:58,  Date: 2021-07-04
BUYING: Day:59,  Date: 2021-07-05
SELLING: Day:60,  Date: 2021-07-06
BUYING: Day:61,  Date: 2021-07-07
SELLING: Day:62,  Date: 2021-07-08
BUYING: Day:63,  Date: 2021-07-09
SELLING: Day:64,  Date: 2021-07-10
BUYING: Day:65,  Date: 2021-07-11
SELLING: Day:66,  Date: 2021-07-12
BUYING: Day:67,  Date: 2021-07-13
SELLING: Day:68,  Date: 2021-07-14
BUYING: Day:69,  Date: 2021-07-15
SELLING: Day:70,  Date: 2021-07-16
BUYING: Day:71,  Date: 2021-07-17
SELLING: Day:72,  Date: 2021-07-18
BUYING: Day:73,  Date: 2021-07-19
SELLING: Day:74,  Date: 2021-07-20
BUYING: Day:75,  Date: 2021-07-21
SELLING: Day:76,  Date: 2021-07-22
BUYING: Day:77,  Date: 2021-07-23
